In [1]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier



# Conexão com o banco de dados PostgreSQL
conexao = psycopg2.connect(host="localhost", database="Projeto_Final", user="postgres", password="", port=5432)

# Consulta SQL para selecionar os dados do banco de dados
consulta_sql = """
SELECT cl."cliente", 
       c."IDCREDITO", 
       hc."HISTORICO", 
       p."PROFISSAO",
       f."FIADOR",
       i."INVESTIMENTOS",
       of."OUTROSFINANCIAMENTOS",
       c."Status"
FROM public."CLIENTES" cl
JOIN public."CREDITO" c ON cl."idcliente" = c."IDCREDITO"
JOIN public."HISTORICO_CREDITO" hc ON c."HistoricoCredito" = hc."IDHISTCRED"
JOIN public."PROFISSAO" p ON c."Profissao" = p."IDPROFISSAO"
JOIN public."FIADOR" f ON c."FiadorTerceiros" = f."IDFIADOR"
JOIN public."INVESTIMENTOS" i ON c."Investimentos" = i."IDINVESTIMENTOS"
JOIN public."OUTROSFINANC" of ON c."OutrosFinanciamentos" = of."IDOUTROSFINANC";
"""

# Carregar os dados do banco de dados para um DataFrame
df = pd.read_sql_query(consulta_sql, conexao)
df.shape

/var/folders/_k/58jdphbs5pj0tbmhzg3ccj5w0000gn/T/ipykernel_43613/3654625468.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(consulta_sql, conexao)


(241, 8)

In [2]:
previsores = df.iloc[:, 0:7].values
classe = df.iloc[:,7].values

In [3]:
# Transformação dos atributos categóricos em atributos numéricos
labelencoder1 = LabelEncoder()
previsores[:,0] = labelencoder1.fit_transform(previsores[:,0])

labelencoder2 = LabelEncoder()
previsores[:,2] = labelencoder2.fit_transform(previsores[:,2])

labelencoder3 = LabelEncoder()
previsores[:, 3] = labelencoder3.fit_transform(previsores[:, 3])

labelencoder4 = LabelEncoder()
previsores[:, 4] = labelencoder4.fit_transform(previsores[:, 4])

labelencoder5 = LabelEncoder()
previsores[:, 5] = labelencoder5.fit_transform(previsores[:, 5])

labelencoder6 = LabelEncoder()
previsores[:, 6] = labelencoder6.fit_transform(previsores[:, 6])


In [4]:
# Divisão da base de dados entre treinamento e teste. Usamos 30% para testar e 70% para treinar. Random_state = 0 para sempre obter a mesma divisão da base quando o código for executado
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(previsores,
                                                                  classe,
                                                                  test_size = 0.3,
                                                                  random_state = 0)

In [5]:
# Criação do modelo, treinamento
floresta = RandomForestClassifier(n_estimators = 100)
floresta.fit(X_treinamento, y_treinamento)

RandomForestClassifier()

In [6]:
# Visualização dos atributos principais
floresta.estimators_

[DecisionTreeClassifier(max_features='sqrt', random_state=1545470041),
 DecisionTreeClassifier(max_features='sqrt', random_state=610205114),
 DecisionTreeClassifier(max_features='sqrt', random_state=1365690879),
 DecisionTreeClassifier(max_features='sqrt', random_state=1646575849),
 DecisionTreeClassifier(max_features='sqrt', random_state=1630368091),
 DecisionTreeClassifier(max_features='sqrt', random_state=102423230),
 DecisionTreeClassifier(max_features='sqrt', random_state=1594478564),
 DecisionTreeClassifier(max_features='sqrt', random_state=1823210592),
 DecisionTreeClassifier(max_features='sqrt', random_state=642322638),
 DecisionTreeClassifier(max_features='sqrt', random_state=1209611275),
 DecisionTreeClassifier(max_features='sqrt', random_state=1745372224),
 DecisionTreeClassifier(max_features='sqrt', random_state=1930039951),
 DecisionTreeClassifier(max_features='sqrt', random_state=1408403985),
 DecisionTreeClassifier(max_features='sqrt', random_state=2094933818),
 Decision

In [7]:
previsoes = floresta.predict(X_teste)
confusao = confusion_matrix(y_teste, previsoes)
taxa_acerto = accuracy_score(y_teste, previsoes)
print(taxa_acerto.round(1)*100,"%")

80.0 %


In [8]:
#matriz de confusão
confusao = confusion_matrix(y_teste, previsoes)
confusao

array([[ 7, 11],
       [ 7, 48]])